# Presentación

La práctica ha sido realizada por el grupo L5, que esta compuesto por:

*   Borja Arán Tejada 100363921
*   Adrián Vázquez Pérez 100363906
*   Álvaro Iglesias García 100363759
*   Antonio Rodríguez Arias 100363883




# Introducción

En este cuaderno Jupyter hemos implemetado la práctica 2.2 de la asignatura RAI, el cual hemos dividido en los siguientes apartados:
  *  **Imports necesarios**: en este puntos importamos todas las librerías que utilizamos para el desarrollo de la práctica.
  *  **Funciones auxiliares**: en este apartado hemos implementado todas las funciones necesarias para llevar a cabo la limpieza de los htmls y las queries, el cálculo de la matriz TFIDF y todos aquellos métodos auxiliares que hemos precisado para el cálculo de las métricas.
  *  **Métricas**: apartado donde implementamos las métricas precisadas en el enunciado de la práctica.
  *  **Ejecución principal**: aquí se ha implementado el hilo principal de ejecución que llama a todos los métodos del cuaderno Jupyter y muestra por pantalla el resultado de las métricas implementadas.

A lo largo de este cuaderno Jupyter, se explicará detallademente en cada uno de los apartados y subapartados del índice, cuales son las funcionales implementadas en cada uno de los puntos, así como, la justificación de las decisiones de diseño en la propia implementación.

# Imports necesarios 

En este apartado están todas las librerías importadas que necesitamos para el desarrollo del proyecto

In [ ]:
# Librería para sincronizar con Google Drive
from google.colab import drive

# Librerias para manejar archivos
import urllib
import pickle
import re
import string
import os
import xml.etree.ElementTree

# Librerias para la limpieza de texto(normalización, tokenización, stopwords, etc)
from bs4 import BeautifulSoup
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tag import pos_tag
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize

# Librerias para el cálculo de la matriz TFIDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Librería para el cálculo de logaritmos en base 2
import math

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# Funciones auxiliares

## Funciones de limpieza utilizadas en la práctica 2.1

En primer lugar se ha desarrollado un método "limpieza" que implementa todas las funcionalidades necesarias para limpiar el texto de los documentos proporcionados. En este método se realiza una llamada a todos los métodos auxiliares creados en la práctica 2.1 con el fin de obtener un array que contenga todas las palabras limpias de cada uno de los documentos (htmls y topics). Los diferentes métodos auxiliares que hemos desarrollado para implementación de la limpieza son:
 *  **get_text**: recibe un documento como parámetro y tiene como objetivo la eliminación de tags html
 *  **remove**: recibe un documento como parámetro y tiene como objetivo la simplificación del texto y la eliminación de caracteres especiales.
 * **lower_case**: convierte todas las palabras del corpus de un documento a minúsculas.
 * **delete_stopwords**: elimina todas las palabras sin valor (palabras vacías) del documento.
 * **lemmatiza_texts**: método que lematiza el texto del documento que se introduce como parámetro con el objetivo de obtener la raíz léxica de cada una de las palabras del corpus. Antes de llevar a cabo el proceso de lematización, se hace una llamada **post_tagger**, con el cual se le añade un tag a cada una de las palabras (tokenización) con el fin de saber que tipo de palabras son y facilitar la propia lematización.

Una vez que el método **limpieza** se ha ejecutado para todos los documentos html y xml (queries) tendremos todas las palabras limpias y listas para poder operar con ellas.

In [ ]:
def limpieza(corpus):
  corpus = get_text(corpus)
  corpus = remove(corpus)
  corpus = lower_case(corpus)
  corpus = delete_stopwords(corpus)
  corpus = lemmatize_texts(corpus)
  return corpus


def get_text(corpus):
  soup = BeautifulSoup(corpus, "html.parser")
  corpus =soup.get_text()
  return corpus

def remove(corpus):
  corpus = corpus.replace('\n', ' ')
  corpus = corpus.replace('\t', ' ')
  corpus = corpus.replace('-', ' ')
  corpus = re.sub('<.*?>', " ", corpus)
  corpus = re.sub('{.*?}', " ", corpus)
  corpus = re.sub(r"\([^()]*\)", " ", corpus)
  corpus = corpus.translate(str.maketrans("","", string.punctuation))
  corpus = re.sub(' +', ' ',corpus)
  return corpus

def lower_case(corpus):
  return corpus.lower()

def delete_stopwords(body):
  clean = ""
  stop_words = set(nltk.corpus.stopwords.words('english'))
  word_tokens = nltk.tokenize.word_tokenize(body)
  for j in word_tokens:
    if not j in stop_words:
      clean = clean + " " +str(j)
  return clean

def lemmatize_texts(corpus):
  clean = ""
  lemm = WordNetLemmatizer()
  tokens = pos_tag(word_tokenize(corpus))
  for word, tag in tokens:
    if pos_tagger(tag) is None:
      clean = clean + " " + str(word)
    else:
      clean = clean + " " +lemm.lemmatize(word, pos_tagger(tag))
  return clean

def pos_tagger(nltk_tag):
  if nltk_tag.startswith('J'):
      return wordnet.ADJ
  elif nltk_tag.startswith('V'):
      return wordnet.VERB
  elif nltk_tag.startswith('N'):
      return wordnet.NOUN
  elif nltk_tag.startswith('R'):
      return wordnet.ADV
  else:          
      return None

## Cálculo de la matriz TFIDF

Consideramos que el cálculo a través de la función de similitud coseno TFIDF nos da los valores más fiables para aproximar la relevancia de cada documento para cada una de las queries.

Lo que hacemos en este apartado para calcular la matriz TFIDF es:
* Calculamos la metriz de pesos del corpus introducido por argumento (tfidf_vectorizer.fit_transform(corpus)).
* Vectorizamos las queries introducidas por argumento (tfidf_vectorizer.transform(queries)).
* Calculamos la función de similitud (cosine_similarity(queries_tfidf, matriz_tfidf)).
* Transponemos la matriz TFIDF para obtener los resultados en un formato más accesible.

In [ ]:
def matriz_TFIDF(corpus,queries):
  
  tfidf_vectorizer = TfidfVectorizer()

  matriz_tfidf = tfidf_vectorizer.fit_transform(corpus)

  queries_tfidf = tfidf_vectorizer.transform(queries)

  tfidf_matriz = cosine_similarity(queries_tfidf, matriz_tfidf)

  matriz = np.transpose(tfidf_matriz)

  return matriz

## Ordenación de mayor a menor de los documentos por su valor para cada consulta

El método **relevantes_ordenados** se encarga de devolver los documentos relevantes ordenados (de mayor a menor) según los valores obtenidos en la matriz TFIDF, con el objetivo de obtener primero aquellos documentos que tengan mayor relevancia para cada una de los topics. 

El método recibe como argumento:
* El índice de la consulta (**index**)
* El umbral: es el valor mínimo que hemos considerado que un documento debe de tener para ser relevante para una query, según la matriz TFIDF.
* La matriz TFIDF (**matriz**)
* Array que contiene el nombre de todos los documentos htmls (**docs**)

Este método empieza guardando en el array ordenados todos los valores de la matriz TFIDF, después los ordena de mayor a menor, y por último, se guarda en un diccionario (dic) el nombre del documento de aquellos que superen o igualen el umbral que hemos indicado (0,09).

In [ ]:
def relevantes_ordenados(index, umbral, matriz, docs):
  dic = []
  ordenados = []
  for i in range(len(matriz)):
    ordenados.append(matriz[i][(index)])
  ordenados.sort(reverse=True)
  for i in range(len(ordenados)):
    if ordenados[i] >= umbral:
      for j in range(len(matriz)):
        if matriz[j][(index)]==ordenados[i]:
          dic.append(docs[j])
  return dic


## Funciones auxiliares para el cálculo de métricas

En este apartado del cuaderno Jupyter contiene las siguientes funciones que nos ayudarán a calcular las métricas:
* **total_relevantes**: Función que cuenta el total de documentos relevantes recuperados para una query (utilizado en la métrica de exhaustividad).
* **check_trel**: Función que devuelve la relevancia de un documento html respecto a un topic del archivo union.trel (utilizado en todas las métricas).
* **lista_relevantes_ordenados**: Función que obtiene la lista ideal de la relevancia de los documentos según la query, ordenada de mayor a menor (utilizado en la métrica nDGC). 
* **todas_las_metricas**: Función que llama a todos los métodos que implementan las diferentes métricas solicitadas para una query. Además, imprime por pantalla todos los resultados obtenidos. 

In [ ]:
def total_relevantes(ruta_trel, index):
  res = 0
  with open(ruta_trel, "r") as trel:
    for line in trel:
      if (line.split()[0]==str(index) and int(line.split()[2])> 0):
        res += 1
    return res

def check_trel(ruta_trel, html, index):
  with open(ruta_trel, "r") as trel:
    for line in trel:
      if ((line.split()[1]+".html")==html and line.split()[0]==str(index)):
        return int(line.split()[2])
  return 0

def lista_relevantes_ordenados (index, recuperados, ruta_trel):
  lista = []
  for file in recuperados[index]:
      if (check_trel(ruta_trel, file, index+1)) >=1:
         lista.append(check_trel(ruta_trel, file, index+1))
  lista.sort(reverse=True)
  return lista
  
def todas_las_metricas(index, recuperados, ruta_trel, p , r, f , rr_1, rr_2, ap, ndcg_1, ndcg_2):
  p.append(metrica_precision(index, recuperados, ruta_trel))
  r.append(metrica_exhaustividad(index, recuperados, ruta_trel))
  f.append(valor_f(p[index], r[index]))
  rr_1.append(metrica_reciproca(index, recuperados, ruta_trel, 1))
  rr_2.append(metrica_reciproca(index, recuperados, ruta_trel, 2))
  ap.append(average_precision(index, recuperados, ruta_trel))
  ndcg_1.append(NDGC(index, recuperados, ruta_trel, 10))
  ndcg_2.append(NDGC(index, recuperados, ruta_trel, 100))
  print("PRECISION: " + str(p[index]))
  print("RECALL: " + str(r[index]))
  print("VALOR-F: " + str(f[index]))
  print("RECIPROCA 1 : "+ str(rr_1[index]))
  print("RECIPROCA 2 : "+ str(rr_2[index]))
  print("AVERAGE PRECISION : "+ str(ap[index]))
  print("nDCG CORTE 10: " + str(ndcg_1[index]))
  print("nDCG CORTE 100: " + str(ndcg_2[index]))
  print()
  print("/////////////////////////////////////////////////////////////////////////////////////")
  print()

# Métricas

## Precisión

**Precisión = |recuperados ∩ relevantes|/|recuperados|**

Con esta métrica averiguamos que porcentaje de los documentos recuperados para una query son relevantes.

Para esta métrica hemos decidido recorrer el array de lo documentos recuperados para una query en cuestión, y contar aquellos que sean relevantes para cortes de 5 y 10 documentos. Después, el número de documentos relevantes recuperados se divide por el total de recuperados, pero al tratarse de cortes de 5 y 10, se dividirá por 5 o 10 respectivamente. Al final devolvemos el resultado en conjunto para ambos cortes, una vez por cada una de las queries especificadas.

In [ ]:
def metrica_precision(index, recuperados, ruta_trel):
  precision_corte_5 = 0
  precision_corte_10 = 0

  cont = 0
  for file in recuperados[index]:
    if cont < 5 and (check_trel(ruta_trel, file, index+1)) >= 1:
      precision_corte_5 += 1
    if cont < 10 and (check_trel(ruta_trel, file, index+1)) >= 1:
      precision_corte_10 += 1
    cont += 1

  if (len(recuperados[index]) > 0):
    precision_corte_5 = precision_corte_5 / 5
    precision_corte_10 = precision_corte_10 / 10
  return {"Precision 5": round(precision_corte_5, 4), "Precision 10": round(precision_corte_10, 4)}



## Exhaustividad

**Exhaustividad = |recuperados ∩ relevantes|/|relevantes|**

Con esta métrica averiguamos que porcentaje de los documentos relevantes para una query son recuperados.

Para esta métrica contamos para los 5 y 10 (cortes especificados) primeros documentos recuperados aquellos que sean relevantes y después, dividimos ese valor obtenido por el total de relevantes que se hayan recuperado para una query en concreto. El número total de documentos relevantes recuperados para dicha query nos la proporciona la función anteriormente explicada **total_relevantes**, y tal y como sucede con la métrica de preción, devolvemos los resultados en conjunto para los cortes de 5 y 10 solicitados para cada una de las queries.

In [ ]:
def metrica_exhaustividad(index, recuperados, ruta_trel):
  precision_corte_5 = 0
  precision_corte_10 = 0
  contadorRelevantes = 0

  cont = 0
  for file in recuperados[index]:
    if cont < 5 and (check_trel(ruta_trel, file, index+1)) >= 1:
      precision_corte_5 += 1
    if cont < 10 and (check_trel(ruta_trel, file, index+1)) >= 1:
      precision_corte_10 += 1
    cont += 1

  relevantes = total_relevantes(ruta_trel, index+1)

  if (relevantes > 0):
      precision_corte_5 = precision_corte_5 / relevantes
      precision_corte_10 = precision_corte_10 / relevantes
  return {"Precision 5": round(precision_corte_5, 4), "Precision 10": round(precision_corte_10, 4)}

## Valor-F


**F = 2xPrecisiónxExhaustividad/Precisión+Exhaustividad**

Con esta métrica podemos implementar una medida estadística de precisión de nuestro motor de recuperación.

En esta métrica se pide por argumento la precisión y exhaustividad para los distintos cortes (5 y 10). Con el conjuntos de datos de precisión y exhaustividad obtenidos en las métricas anteriores, se aplica la fórmula especificada anteriormente para aquellos casos donde Precisión+Exhaustividad sea distinto de 0 y se devuelve el resultado obtenido del cálculo (en cortes de 5 y 10 para cada query), en el caso contrario se devuelve un 0 como resultado.


In [ ]:
def valor_f ( p , r):
  f_5 = 0
  f_10 = 0
  if (p["Precision 5"] + r["Precision 5"]>0):
    f_5 = (2*p["Precision 5"]*r["Precision 5"])/(p["Precision 5"]+r["Precision 5"])
  if (p["Precision 10"] + r["Precision 10"]>0):
    f_10 = (2*p["Precision 10"]*r["Precision 10"])/(p["Precision 10"]+r["Precision 10"])
  return {"Precision 5": round(f_5, 4), "Precision 10": round(f_10, 4)}

## Reciprocal Rank

**RR = 1/Rank**

**Donde Rank es la posición del primer documento relevante recuperado.**

El rango recíproco es inverso al rango del primer documento relevante recuperado. Por ello, para implementar esta métrica simplemente recorremos todos los documentos recuperados y devolvemos 1/(i+1), donde i es la posición del primer documento relevante (se le suma uno para obtener la posición real y evitar la posición 0). Para cada query, se obtendrán dos rango recíprocos, teniendo en cuenta una relevancia mínima de 1 (**param=1**) y otro caso con relevancia mínima 2 (**param=2**), donde se tendrá en cuenta la posición del primer documento relevante que cumpla dicho requísito.

In [ ]:
def metrica_reciproca(index, recuperados, ruta_trel, param):
  i = 0
  for file in recuperados[index]:
    if (check_trel(ruta_trel, file, index+1)>=param):
      return round(1/(i+1), 4)
    i= i+1
  return 0

## Average Precision

**AP = X/|relevantes recuperados|**

**Donde X es Σ(posición de relevante/posición en la recuperación de documentos) ∀ todo documento relevante recuperado.**

Average Precision es la media de precisión después de recuperar cada documento relevante, mide el área bajo la curva precisión-exhaustividad. 

Para la implementación de esta métrica hemos tenido que dividir el cálculo en dos partes. En primer lugar, calculamos el numerador de la fórmula fraccionaria, donde hacemos el sumatorio de la posición de cada uno de los documentos relevantes respecto a los otros documentos relevantes recuperados, y cada una de las posiciones de los documentos relevantes se divide por su posición respecto a todos los documentos recuperados. El resultado de este cálculo se almacenará en la variable **numerador**, la posición de cada uno de los documentos relevantes se guardará en **total_relevantes**, y la posición de dichos documentos relevantes respecto a todos los documentos recuperados en **i**. Por último, devolvemos como resultado la división de **numerador** entre **total_relevantes**, pudiendo haber como máximo 100 documentos recuperados para el cálculo de la precisión media de cada query.

In [ ]:
def average_precision(index, recuperados, ruta_trel):
  res = 0
  i = 1
  numerador = 0
  total_relevantes = 0
  for file in recuperados[index]:
    if total_relevantes<=100 and (check_trel(ruta_trel, file, index+1)>=1):  
      total_relevantes+=1
      numerador = numerador  + total_relevantes/i
    i = i+1
  if (total_relevantes>0):
    res = numerador/total_relevantes
    return round(res,4)
  else:
    return 0

## Normalized Discounted Cumulated Gain

**nDCG = DCG/DCGi**

**Donde DGC es Discounted Cumulated Gain y DCGi es lo mismo, pero el caso que procede de llevar a cabo los cálculos con la recuperación ideal.**

En primer lugar empezamos calculando la ganancia de información relevante, tanto para el caso normal como para el ideal(**G** y **Gi**). Para calcular **G** obtenemos la relevancia de los primeros documentos (incluyendo los que no tienen relevancia) para los diferentes cortes especificados (10 y 100), sin embargo, para el cálculo de **Gi** almacenamos los documentos relevantes ordenados de mayor a menor según su relevancia. La relevancia de los documentos para cada una de las queries se obtienen a partir del archivo **union.trel**.

El siguiente cálculo necesario es el Cumulated Gain (**CG** y **CGi**), para el cual hemos aplicado la siguiente fórmula:
- Si es la primera posición, el valor de **CG[0]** es igual a **G[0]**.
- Si la posición actual es mayor que la primera posición, el valor de **CG[posición actual]** es igual a **G[posición actual]+G[posición anterior]**.

Para el caso ideal  (**CGi**) se calcula de la misma forma que **CG**, sin embargo, si la dimensión de **CGi** es menor que la de **CG**, rellenamos el array de **CGi** con **CGi[posición actual] es igual a CGi[posición anterior]**, hasta que ambos tengan una dimensión similar.

Posteriormente es necesario el Discounted Cumulated Gain (**DCG** y **DCGi**), pero para este cálculo el caso ideal se calcula de la misma manera que el caso normal, puesto que **CG** y **CGi** tienen la misma dimensión. Para obtener **DCG** y **DCGi** aplicamos la siguiente fórmula, teniendo en cuenta que operamos en base 2 (**b=2**):
- Si la posición es menor que **b** el valor de **DCG[posición actual]** es igual a **CG[posición actual]**.
- Si la posición actual es mayor o igual que **b**, el valor de **DCG[posición actual]** es igual a **DCG[posición anterior]+G[posición actual]/log2(posición actual)**.

Finalmente para obtener **nDCG_Array** dividiremos, para cada consulta, cada valor obtenido de **DCG** entre cada valor obtenido de **DCGi**, retornando como resultado final un array para cada corte (dimensión 10 y dimensión 100). Por otro lado, también calcularemos para cada consulta y corte especificado, el nDCG total (**nDCG**), el cual hemos obtenido aplicando la siguiente fórmula:
**nDCG = Σ(DCG)/Σ(DCGi)**

In [ ]:
def NDGC(index, recuperados, ruta_trel, corte):
  G = []
  CG = []
  Gi = []
  CGi = []
  DCG = []
  DCGi = []
  nDCG = []
  nDCG_numerador = 0
  nDCG_denominador = 0
  aux = 0
  auxi = 0
  lista_ordenados = lista_relevantes_ordenados (index, recuperados, ruta_trel)

  for file in recuperados[index]:
    if aux< corte:
      G.append(check_trel(ruta_trel, file, index+1))
      if aux < len(lista_ordenados):
        Gi.append(lista_ordenados[aux])
      aux= aux+1

  aux = 0
  for aux in range(len(G)):
    if aux > 0:
      CG.append(G[aux]+ CG[aux-1])
      if aux < len(Gi):
        CGi.append(Gi[aux]+ CGi[aux-1])
      else:
        CGi.append(CGi[aux-1])
    else:
      CG.append(G[aux])
      if aux == len(Gi):
        CGi.append(0)
      else:
        CGi.append(Gi[aux])

    if aux < 2:
      DCG.append(CG[aux])
      DCGi.append(CGi[aux])
    else:
      DCG.append(DCG[aux-1]+G[aux]/math.log2(aux))
      if aux < len(Gi):
        DCGi.append(DCGi[aux-1]+Gi[aux]/math.log2(aux))
      else:
        DCGi.append( DCGi[aux-1])

    if DCGi[aux] == 0:
      nDCG.append(0)
    else:
      nDCG_numerador = nDCG_numerador + DCG[aux]
      nDCG_denominador = nDCG_denominador + DCGi[aux]
      nDCG.append(round(DCG[aux]/DCGi[aux],4))
      
  return {"nDCG": round(nDCG_numerador/nDCG_denominador, 4), "nDCG_Array": nDCG}


# Ejecución principal

Variables globales:
- **docs**: variable donde almacenamos los nombres de todos los documentos.
- **html**: variable auxiliar para poder acceder al contenido de un documento, con el fin de limpiarlo usando la librería *open*.
- **corpus**: lista con el contenido de todos los documentos limpios.
- **queries**:lista con el contenido de todas los queries limpias.
- **recuperados**:lista con los nombres de los documentos ordenados de mayor a menor, según su relevancia en la matriz TFIDF para cada consulta.
- **umbral**: relevancia mínima que debe tener un documento para poder almacenarse en la lista de **recuperados**. El valor de esta variable ha sido determinado después de exhaustivas pruebas, siendo el mejor valor para filtrar sin eliminar demasiados documentos relevantes.

In [ ]:
#Conexión con google drive
drive.mount('/content/drive')
#Variable con la dirección de los documentos html
path_docs = "/content/drive/MyDrive/RAI/Practica2/2.2/documents.biased"
#Variable con la dirección del archivo topics.xml
ruta_topics = "/content/drive/MyDrive/RAI/Practica2/2.2/topics.xml"
#Variable con la dirección del archivo union.trel
ruta_trel = "/content/drive/MyDrive/RAI/Practica2/2.2/union.trel"

docs = []
html = []
corpus = []
queries = []
recuperados = []

umbral = 0.09

#Accedemos a los documentos html y los limpiamos
i = 0
for dirpath, dirnames, filenames in os.walk(path_docs):
  for filename in filenames:
    docs.append(filename)
    html.append(open (os.path.join(dirpath, filename), 'r'))
    corpus.append(limpieza(html[i]))
    i = i+1

#Accedemos al archivo topics.xml y preparamos las consultas
ubicacion = xml.etree.ElementTree.parse(ruta_topics).getroot()
for query in ubicacion.findall('topic'):
  queries.append(limpieza(query.find('title').text))

#Cálculo de la matriz TFIDF
matriz = matriz_TFIDF(corpus,queries)

Mounted at /content/drive


In [ ]:
#Ordenar y printea los documentos recuperados en orden para cada consulta
i=0
for query in queries:
  print("Consulta "+ str(queries.index(query)+1))
  recuperados.append(relevantes_ordenados(int(queries.index(query)), umbral, matriz, docs))
  if len(recuperados[i]) > 0:
    print(recuperados[i])
  else:
    print("La consulta "+str(queries.index(query)+1)+" no tiene documentos relevantes")
  print()
  i = i+1


In [ ]:
#Printeamos y calculamos métricas
p = []
r = []
f = []
rr_1 = []
rr_2 = []
ap = []
ndcg_1 = []
ndcg_2 = []
for i in range(len(queries)):
  print("Consulta "+ str(i+1))
  todas_las_metricas(i, recuperados, ruta_trel, p , r, f, rr_1, rr_2, ap, ndcg_1, ndcg_2)

Consulta 1
PRECISION: {'Precision 5': 0.4, 'Precision 10': 0.5}
RECALL: {'Precision 5': 0.0323, 'Precision 10': 0.0806}
VALOR-F: {'Precision 5': 0.0598, 'Precision 10': 0.1388}
RECIPROCA 1 : 0.25
RECIPROCA 2 : 0.027
AVERAGE PRECISION : 0.6337
nDCG CORTE 10: {'nDCG': 0.1911, 'nDCG_Array': [0.0, 0.0, 0.0, 0.112, 0.1845, 0.238, 0.2247, 0.2625, 0.2947, 0.283]}
nDCG CORTE 100: {'nDCG': 0.5487, 'nDCG_Array': [0.0, 0.0, 0.0, 0.112, 0.1845, 0.238, 0.2247, 0.2625, 0.2947, 0.283, 0.3092, 0.3326, 0.3537, 0.3729, 0.3624, 0.3794, 0.3951, 0.4098, 0.4234, 0.4363, 0.4483, 0.4391, 0.4503, 0.4416, 0.452, 0.4619, 0.4538, 0.4631, 0.472, 0.4644, 0.4728, 0.4809, 0.4887, 0.4815, 0.4889, 0.4821, 0.5029, 0.5229, 0.5292, 0.5224, 0.5284, 0.5342, 0.5277, 0.5214, 0.527, 0.5325, 0.5265, 0.5206, 0.5258, 0.5309, 0.5359, 0.5465, 0.557, 0.5674, 0.5778, 0.5778, 0.5778, 0.5778, 0.5778, 0.5778, 0.5778, 0.5879, 0.5879, 0.5979, 0.6078, 0.6178, 0.6178, 0.6276, 0.6375, 0.6472, 0.6472, 0.657, 0.657, 0.657, 0.6666, 0.6666, 0.66

In [ ]:
p_total_5 = 0
p_total_10 = 0
r_total_5 = 0
r_total_10 = 0
f_total_5 = 0
f_total_10 = 0
rr_1_total = 0
rr_2_total = 0
ap_total = 0
ndcg_1_total = 0
ndcg_2_total = 0
for i in range (len(queries)):
  p_total_5 = p_total_5 + p[i]["Precision 5"]
  p_total_10 = p_total_10 + p[i]["Precision 10"]
  r_total_5 = r_total_5 + r[i]["Precision 5"]
  r_total_10 = r_total_10 + r[i]["Precision 10"]
  f_total_5 = f_total_5 + f[i]["Precision 5"]
  f_total_10 = f_total_10 + f[i]["Precision 10"]
  rr_1_total = rr_1_total + rr_1[i]
  rr_2_total = rr_2_total + rr_2[i]
  ap_total = p_total_5 + ap[i]
  ndcg_1_total = ndcg_1_total  + ndcg_1 [i]["nDCG"]
  ndcg_2_total = ndcg_2_total  + ndcg_2 [i]["nDCG"]

p_total_5 = round((p_total_5/len(queries)),4)
p_total_10 = round((p_total_10/len(queries)),4)
r_total_5 = round((r_total_5/len(queries)),4)
r_total_10 = round((r_total_10/len(queries)),4)
f_total_5 = round((f_total_5/len(queries)),4)
f_total_10 = round((f_total_10/len(queries)),4)
rr_1_total = round((rr_1_total/len(queries)),4)
rr_2_total = round((rr_2_total/len(queries)),4)
ap_total = round((ap_total/len(queries)),4)
ndcg_1_total = round((ndcg_1_total/len(queries)),4)
ndcg_2_total = round((ndcg_2_total/len(queries)),4)

print("Medias de las métricas del motor de evaluación:")
print("-------------------------------------------")
print("Precision en corte 5 = "+ str(p_total_5))
print("Precision en corte 10 = "+ str(p_total_10))
print("Exhaustividad en corte 5 = "+ str(r_total_5))
print("Exhaustividad en corte 10 = "+ str(r_total_10))
print("Valor-F en corte 5 = "+ str(f_total_5))
print("Valor-F en corte 10 = "+ str(f_total_10))
print("Recriprocal Rank (1) = "+ str(rr_1_total))
print("Reciprocal Rank (2) = "+ str(rr_2_total))
print("Average Precision = "+ str(ap_total))
print("nDCG en corte 10 = "+ str(ndcg_1_total))
print("nDCG en corte 100 = "+ str(ndcg_2_total))

Medias de las métricas del motor de evaluación:
-------------------------------------------
Precision en corte 5 = 0.6133
Precision en corte 10 = 0.6
Exhaustividad en corte 5 = 0.0504
Exhaustividad en corte 10 = 0.1018
Valor-F en corte 5 = 0.0928
Valor-F en corte 10 = 0.1727
Recriprocal Rank (1) = 0.7159
Reciprocal Rank (2) = 0.4228
Average Precision = 0.6487
nDCG en corte 10 = 0.442
nDCG en corte 100 = 0.5835


# Conclusión

Los resultados de las metricas desarrolladas son válidos, debido a que ninguno es un resultado incongruente, estando siempre en un intervalo lógico. Además, los resultados de varias consultas han sido calculados manualmente y coinciden los valores con los mostrados por pantalla, lo cual significa que el motor de búsqueda desarrollado es preciso, ya que consigue satisfacer cada consulta con los documentos recuperados, calculados por el motor.

Para los resultados de las consultas 5 y 11, procedemos a explicar el por qué de los resultados tan diferentes con respecto a las otras consultas. Estos resultados se deben a la existencia de términos polisémicos en la consulta, lo que genera ruido en la recuperación del motor. En la consulta 5 este término es *Apple*, ya que al prepararlo para ser vectorizado, nos es imposible determinar que se refiere a la empresa tecnológica o que es interpretado como una jugosa fruta. Y para la consulta 11, el término es *Kensington*, que en lugar de ser detectado como el tipo de cable, puede ser interpretado como el centro de estudio, como el barrio londinense, etc.